## **COMO RODAR?**

Dentro de cada célula, apertar *Ctrl+Enter* ou clicar em *Run* - ícone superior da página.

**Observação 1:** 
- Rodar as células em ordem. A última célula terá o resultado que deseja. 

Caso queira rodar todas as células de uma vez: cell -> Run All

**Observação 2:** 
- Qualquer modificação em uma célula, todas devem ser rodadas novamente e em ordem. 

### Instalações necessárias antes de utilizar este módulo. 

    ! pip install nltk
    nltk.download()
    ! pip install deep_translator


In [8]:
import nltk
from itertools import permutations
from deep_translator import GoogleTranslator

---------
**Tradução da sentença em inglês**

**Por que traduzir?**

    Porque a maioria das bilbiotecas para tokenizar e para encontrar a morfologia são associadas à lingua inglesa. Isso otimiza o código, evitando que ele trave dependendo do computador.

In [9]:
sentenca = 'A Segunda Guerra Mundial, conflito militar entre 1939 a 1945, envolveu as maiores potências da época'

traduzir_ingles  = GoogleTranslator(source='portuguese', target='english').translate(sentenca)
print(f'Tradução para o inglês: {traduzir_ingles}')

Tradução para o inglês: The Second World War, a military conflict between 1939 and 1945, involved the greatest powers of the time.


-------
**Tokenização**

In [10]:
tokens = nltk.word_tokenize(traduzir_ingles)
if '.' in tokens:
    tokens.remove('.')
#Lista de tuplas cujos elementos são respectivamente a palavra e sua morfologia
tagged = nltk.pos_tag(tokens) 
display(tagged)

[('The', 'DT'),
 ('Second', 'JJ'),
 ('World', 'NNP'),
 ('War', 'NNP'),
 (',', ','),
 ('a', 'DT'),
 ('military', 'JJ'),
 ('conflict', 'NN'),
 ('between', 'IN'),
 ('1939', 'CD'),
 ('and', 'CC'),
 ('1945', 'CD'),
 (',', ','),
 ('involved', 'VBD'),
 ('the', 'DT'),
 ('greatest', 'JJS'),
 ('powers', 'NNS'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('time', 'NN')]

-----------
**Primeira função a ser chamada**

Separa sujeito (inclui o aposto) do predicado.

Posteriormente, separa o sujeito do aposto caso exista. 

In [4]:
def separa_elementos_oracao (tagged):

    sujeito = []
    predicado = []

    #Separar o sujeito do predicado
    for i,tag in enumerate(tagged):
        if tag[1][0]== 'V':
            break
        else:
            sujeito.append(tag[0])
            
    for j in range(i,len(tagged)):
        predicado.append(tagged[j][0])
    predicado_concatenado = []
    predicado_concatenado.append(' '.join(predicado))

    #Separar o sujeito principal do aposto  
    aposto = []
    sujeito_principal = []

    qnt_virgulas = 0
    posicao_virgulas = []
    for i,tag in enumerate(tagged):
        if tag[0] == ',':
            qnt_virgulas = qnt_virgulas + 1
            posicao_virgulas.append(i)
    
    if len(posicao_virgulas) == 1:
        aposto = sujeito[:posicao_virgulas[0]]
        sujeito_principal = sujeito[posicao_virgulas[0]+1:]
    elif len(posicao_virgulas) == 2:
        aposto = sujeito[posicao_virgulas[0]+1:posicao_virgulas[1]]
        sujeito_principal = sujeito[:posicao_virgulas[0]]+sujeito[posicao_virgulas[1]+1:]
    else:
        sujeito_principal = sujeito
    suj = [] #sujeito em string
    aposto_suj = [] #aposto em string

    suj.append(' '.join(sujeito_principal))
    if len(aposto) != 0:
        aposto_suj.append(' '.join(aposto))
    else:
        aposto_suj.append(None)
        
    return suj[0], aposto_suj[0], predicado_concatenado


-------------
**Permutação entre o sujeito e o aposto**

O aposto da uma característica ao sujeito, logo a frase não tem sentido caso exista a permutação: sujeito + verbo + aposto OU aposto + verbo + sujeito

In [5]:
def permutacao_sujeito_aposto(suj, aposto=None):
    
    if aposto != None:
        primeira_permutacao = list(permutations([suj, aposto]))

        inserir_virgula = []  
        permutacoes_string = []

        #Tranformar tuplas em lista para permitir modificação
        permutacoes = []
        for tupla in primeira_permutacao:
            permutacoes.append(list(tupla))
        
        for i, permutacao in enumerate(permutacoes):
            #A primeira sentenca da permutação começa com letra maiúscula
            if permutacao[0][0] == permutacao[0][0].upper():
                pass
            else:
                lista_palavras = list(permutacao[0])
                lista_palavras[0] = lista_palavras[0].upper()
                permutacao[0] = ''.join(lista_palavras)
            #A segunda sentenca da permutação começa com letra maiúscula    
            if permutacao[1][0] == permutacao[1][0].upper():
                lista_palavras = list(permutacao[1])
                lista_palavras[0] = lista_palavras[0].lower()
                permutacao[1] = ''.join(lista_palavras)
            else:
                pass
                
        # Concatenar strings permutadas
        permutacoes_concatenadas = []
        for i in range(len(permutacoes)):
            permutacoes_concatenadas.append(' '.join(permutacoes[i]))
    
    else:
        permutacoes_concatenadas = [suj]
         
    return permutacoes_concatenadas  

-------------
**Permutação entre as permutações da função anterior com o predicado**

In [6]:
def permutacao_sujeito_predicado(sujeitos, predicado):
    
    primeira_permutacao = []
    for sujeito in sujeitos:
        primeira_permutacao.append(list(permutations([sujeito, predicado[0]])))

    #Tranformar tuplas em lista para permitir modificação
    permutacoes = []
    for lista in primeira_permutacao:
        for tupla in lista:
            permutacoes.append(list(tupla))
            
        for i, permutacao in enumerate(permutacoes):
            #A primeira sentenca da permutação começa com letra maiúscula
            if permutacao[0][0] == permutacao[0][0].upper():
                pass
            else:
                lista_palavras = list(permutacao[0])
                lista_palavras[0] = lista_palavras[0].upper()
                permutacao[0] = ''.join(lista_palavras)
            #A segunda sentenca da permutação começa com letra maiúscula    
            if permutacao[1][0] == permutacao[1][0].upper():
                lista_palavras = list(permutacao[1])
                lista_palavras[0] = lista_palavras[0].lower()
                permutacao[1] = ''.join(lista_palavras)
            else:
                pass
        
        # Concatenar strings permutadas
        permutacoes_concatenadas = []
        for i in range(len(permutacoes)):
    
            if permutacoes[i][-1][-1] == ',':
                frase_com_virgula = list(permutacoes[i][-1])
                frase_com_virgula[-1] = '.'
                permutacoes[i][-1] = ''.join(frase_com_virgula)
            permutacoes_concatenadas.append(' '.join(permutacoes[i]))

    return permutacoes_concatenadas
                

In [11]:
#Nesse caso não há sujeito indeterminado tampouco oculto. 

sujeito, aposto, predicado = separa_elementos_oracao(tagged)
print(f'Sujeito: {sujeito} - Aposto: {aposto}')

if aposto != None:
    permt_suj_apost = permutacao_sujeito_aposto(sujeito, aposto=aposto)

else:
    permt_suj_apost = permutacao_sujeito_aposto(sujeito)

permutacoes = permutacao_sujeito_predicado(permt_suj_apost, predicado)

Sujeito: The Second World War - Aposto: a military conflict between 1939 and 1945


---------
## **Resultado**

In [12]:
for permutacao in permutacoes:
    if permutacao[-1]!= '.':
        permutacao = permutacao + '.'
    permutacao = GoogleTranslator(source='english', target='portuguese').translate(permutacao)
    print(permutacao)

A Segunda Guerra Mundial um conflito militar entre 1939 e 1945 envolveu as maiores potências da época.
Envolveu as maiores potências da época a Segunda Guerra Mundial um conflito militar entre 1939 e 1945.
Um conflito militar entre 1939 e 1945 a Segunda Guerra Mundial envolveu as maiores potências da época.
Envolveu as maiores potências da época um conflito militar entre 1939 e 1945 a Segunda Guerra Mundial.
